In [20]:
import os
os.chdir("C:/Users/Work Test 2/Desktop/AI QA CAD")

In [21]:
import glob
print(glob.glob("ml/data/raw/*.features.csv"))


['ml/data/raw\\01262-24-PAD-R0P4.features.csv', 'ml/data/raw\\sample1.features.csv', 'ml/data/raw\\sample2.features.csv', 'ml/data/raw\\sample3.features.csv']


# Layer Classification Model
This notebook loads feature CSVs generated from CAD drawings, trains a simple classifier to predict the appropriate layer for text entities based on their properties, and saves the trained model.

In [22]:
import glob
import pandas as pd
from pathlib import Path
# Load all feature CSVs from data/raw
data_dir = Path('ml/data/raw')
csv_files = list(data_dir.glob('*.features.csv'))
frames = []
for f in csv_files:
    df = pd.read_csv(f)
    df['source_file'] = f.name
    frames.append(df)
features = pd.concat(frames, ignore_index=True)
features = features.dropna(subset=["TextString", "Layer"])
features.head()

,Handle,ObjType,TextString,Layer,TextHeight,source_file
0,BC3DDF,DBText,1,Z-POINTNUMBER,1.5,01262-24-PAD-R0P4.features.csv
1,BC3DE0,DBText,PLSPK,Z-DESCRIPTION,1.5,01262-24-PAD-R0P4.features.csv
2,BC3DE1,DBText,932.772,Z-ELEVATION,1.5,01262-24-PAD-R0P4.features.csv
3,BC3DE4,DBText,2,Z-POINTNUMBER,1.5,01262-24-PAD-R0P4.features.csv
4,BC3DE5,DBText,FDI_MP,Z-DESCRIPTION,1.5,01262-24-PAD-R0P4.features.csv


In [23]:
# Example preprocessing: use simple heuristics for demo
# Here we will treat 'TextString' as the only feature and the target is 'Layer'
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

X = features['TextString']
y = features['Layer']

model = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression(max_iter=1000))
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model.fit(X_train, y_train)
preds = model.predict(X_test)
print(classification_report(y_test, preds))


                precision    recall  f1-score   support

             0       0.00      0.00      0.00         4
    AS-DRILL-T       0.95      1.00      0.97        18
    AUX-BUFFER       0.00      0.00      0.00         1
      CG-NOTES       0.00      0.00      0.00         1
         D-TXT       1.00      1.00      1.00         3
      DETAIL-T       0.00      0.00      0.00         2
     Defpoints       0.00      0.00      0.00         2
        L-ANNO       0.00      0.00      0.00         1
          L-BD       0.00      0.00      0.00         5
         L-BD1       0.00      0.00      0.00         4
 L-SECTION-LSD       0.00      0.00      0.00         1
        L-SYMB       0.00      0.00      0.00         1
        L-UTIL       0.00      0.00      0.00         1
    P-BUFFER-T       0.00      0.00      0.00         1
P-DRILL-OFFSET       1.00      0.67      0.80         6
       RoadSeg       0.62      1.00      0.77        93
      T-BUSH-T       0.00      0.00      0.00  

c:\Users\Jesse Work\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\Jesse Work\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\Jesse Work\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


In [24]:
# Save the trained model
import joblib
import os
os.makedirs('ml/artifacts', exist_ok=True)
joblib.dump(model, 'ml/artifacts/layer_clf.pkl')
print('Model saved to ml/artifacts/layer_clf.pkl')


Model saved to ml/artifacts/layer_clf.pkl
